In [1]:
import numpy as np
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from plotly import express as px
import plotly.graph_objs as go
# ML libraries - idk which one to use yet
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.patches import Patch
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import preprocessing, tree
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import cKDTree

**where i left off (hailey)**
- TA said to use supervised neural network instead of linear regression so I chose MLPRegressor but its lowkey ass (see below)
- I was thinking predicting relative risk at location/hour/weather would be ideal but idk bc the model isn't looking too hot (this would also require requesting hourly nasa data which i didn't fix get_nasa_power_data to change date to hour)

In [2]:
# NASA API
def get_nasa_power_data(lat, lon, start_date, end_date):
    """
    Fetches NASA POWER API data for given latitude, longitude, and time range.

    Args:
    - lat (float): Latitude of the location.
    - lon (float): Longitude of the location.
    - start_date (str): Start date in YYYYMMDD format.
    - end_date (str): End date in YYYYMMDD format.

    Returns:
    - Pandas DataFrame with selected weather parameters.
    """

    # Specify multiple parameters in the API request
    parameters = "PRECSNO,T2MDEW,PRECTOTCORR,T2M,WS2M"

    url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "parameters": parameters,
        "community": "RE",
        "longitude": lon,
        "latitude": lat,
        "start": start_date,
        "end": end_date,
        "format": "JSON"
    }

    response = requests.get(url, params=params)

    data = response.json()

    # Convert JSON response to DataFrame and transpose it
    nasa_weather = pd.DataFrame.from_dict(data["properties"]["parameter"], orient="index").T

    # Reset index and rename date column
    nasa_weather.reset_index(inplace=True)
    nasa_weather.rename(columns={"index": "date"}, inplace=True)

    # Convert date column to proper datetime format
    nasa_weather["date"] = pd.to_datetime(nasa_weather["date"], format="%Y%m%d", errors="coerce")
    nasa_weather.dropna(subset=["date"], inplace=True)  # Remove invalid date rows

    nasa_weather.rename(columns={
        "PRECSNO": "Snow_Precipitation",
        "T2MDEW": "Dew_Point_2m",
        "PRECTOTCORR": "Total_Precipitation_mm",
        "T2M": "Temperature_2m_C",
        "WS2M": "Wind_Speed_2m"
    }, inplace=True)
    
    # Add Rounded_Lat and Rounded_Lng for merging
    nasa_weather['Rounded_Lat'] = lat
    nasa_weather['Rounded_Lng'] = lon
    
    # Display DataFrame
    print(f"\n Weather Data for Latitude {lat}, Longitude {lon}\n")
    print(f"\n Weather Data for Latitude {lat}, Longitude {lon}\n")
    nasa_weather['Precipitation(in)'] = nasa_weather['Total_Precipitation_mm'] / 25.4 # mm to in
    nasa_weather['Temperature(F)'] = (nasa_weather['Temperature_2m_C'] * (9./5.)) + 32. # C to F
    nasa_weather['Wind_Speed(mph)'] = nasa_weather['Wind_Speed_2m'] * 2.237 # m/s to mph
    # nasa_weather.dropna()
    display(nasa_weather)  # Works in Jupyter Notebook

    return nasa_weather

In [3]:
# Example: Fetch data for different locations
df_la = get_nasa_power_data(34.05, -118.25, "20240101", "20240107")  # Los Angeles


 Weather Data for Latitude 34.05, Longitude -118.25


 Weather Data for Latitude 34.05, Longitude -118.25



,date,Snow_Precipitation,Dew_Point_2m,Total_Precipitation_mm,Temperature_2m_C,Wind_Speed_2m,Rounded_Lat,Rounded_Lng,Precipitation(in),Temperature(F),Wind_Speed(mph)
0,2024-01-01,0.0,6.12,0.04,12.14,1.64,34.05,-118.25,0.001575,53.852,3.66868
1,2024-01-02,0.0,7.05,0.09,11.55,1.59,34.05,-118.25,0.003543,52.790,3.55683
2,2024-01-03,0.0,6.75,5.87,10.86,3.25,34.05,-118.25,0.231102,51.548,7.27025
3,2024-01-04,0.0,2.14,0.02,9.76,2.89,34.05,-118.25,0.000787,49.568,6.46493
4,2024-01-05,0.0,1.99,0.00,10.80,2.00,34.05,-118.25,0.000000,51.440,4.47400
5,2024-01-06,0.0,1.23,0.12,10.50,2.86,34.05,-118.25,0.004724,50.900,6.39782
6,2024-01-07,0.0,-0.09,0.26,7.77,5.29,34.05,-118.25,0.010236,45.986,11.83373


In [4]:
# inspect us_accident data
us_accidents = pd.read_csv('US_Accidents_March23.csv')
us_accidents.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [5]:
unique_values = us_accidents['Weather_Condition'].unique()

print(unique_values)

['Light Rain' 'Overcast' 'Mostly Cloudy' 'Rain' 'Light Snow' 'Haze'
 'Scattered Clouds' 'Partly Cloudy' 'Clear' 'Snow'
 'Light Freezing Drizzle' 'Light Drizzle' 'Fog' 'Shallow Fog' 'Heavy Rain'
 'Light Freezing Rain' 'Cloudy' 'Drizzle' nan 'Light Rain Showers' 'Mist'
 'Smoke' 'Patches of Fog' 'Light Freezing Fog' 'Light Haze'
 'Light Thunderstorms and Rain' 'Thunderstorms and Rain' 'Fair'
 'Volcanic Ash' 'Blowing Sand' 'Blowing Dust / Windy' 'Widespread Dust'
 'Fair / Windy' 'Rain Showers' 'Mostly Cloudy / Windy'
 'Light Rain / Windy' 'Hail' 'Heavy Drizzle' 'Showers in the Vicinity'
 'Thunderstorm' 'Light Rain Shower' 'Light Rain with Thunder'
 'Partly Cloudy / Windy' 'Thunder in the Vicinity' 'T-Storm'
 'Heavy Thunderstorms and Rain' 'Thunder' 'Heavy T-Storm' 'Funnel Cloud'
 'Heavy T-Storm / Windy' 'Blowing Snow' 'Light Thunderstorms and Snow'
 'Heavy Snow' 'Low Drifting Snow' 'Light Ice Pellets' 'Ice Pellets'
 'Squalls' 'N/A Precipitation' 'Cloudy / Windy' 'Light Fog' 'Sand'
 'Snow G

In [ ]:
# list of all columns in US Accident Data
drop_cols = ['ID',
            'Source',
            # 'Severity', # Severity = target column, 1-4, where 1 indicates the least impact on traffic
            'Start_Time',
            'End_Time',
            'Start_Lat',  
            'Start_Lng', 
            'End_Lat',
            'End_Lng',
            'Distance(mi)', # Distance(mi) = target column?, length of road extent affected by accident in miles
            'Description', # Description = human description of accident
            'Street', 
            'City', 
            'County',
            'State',
            'Zipcode',
            'Country',
            'Timezone',
            'Airport_Code',
            'Weather_Timestamp', # Weather_Timestamp = shows time-stamp of weather observation record (in local time)
            # 'Temperature(F)',
            'Wind_Chill(F)',
            'Humidity(%)',
            'Pressure(in)',
            'Visibility(mi)',
            'Wind_Direction',
            # 'Wind_Speed(mph)',
            # 'Precipitation(in)',
            # 'Weather_Condition',
            'Amenity',
            'Bump',
            'Crossing',
            'Give_Way',
            'Junction',
            'No_Exit',
            'Railway',
            'Roundabout',
            'Station',
            'Stop',
            'Traffic_Calming',
            'Traffic_Signal',
            'Turning_Loop',
            'Sunrise_Sunset', # day or night based on sunrise/sunset
            'Civil_Twilight', # day or night based on civil twilight
            'Nautical_Twilight', # day or night based on nautical twilight
            'Astronomical_Twilight'] # day or night based on astronomical twilight

In [7]:
# string feature that if kept will need to be encoded for ML
str_features = 'Weather_Condition'

In [93]:
def prepare_data(df, split, predictors, target):
    """
    Prepares the US Accidents DataFrame for merging with NASA weather data, keeping only necessary columns.
    
    Args:
    - df (DataFrame): Raw US Accidents dataset.
    - split (boolean): if true, split df by target and predictor data
    - ml_drop (list): other columns to drop for machine learning model (adjustable if we decide a variable is not good at predicting)
    
    Returns:
    - DataFrame: Processed DataFrame with 'date', 'Rounded_Lat', 'Rounded_Lng', and 'Severity' columns.
    """
    # Drop duplicates
    df.drop_duplicates(inplace=True)
    
    # Convert time columns to datetime format
    df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')

    # Remove rows with invalid 'Start_Time' values
    df = df[df['Start_Time'].notnull()].copy()

    # Extract 'date' from 'Start_Time' for merging with NASA weather data
    # df['date'] = df['Start_Time'].dt.date

    # Filter for coordinates within LA County
    df = df[(df['Start_Lat'].between(33.7, 34.8)) & (df['Start_Lng'].between(-119.0, -117.6))]

    # Round latitude and longitude to 2 decimal places for approximate matching
    df['Rounded_Lat'] = df['Start_Lat'].round(2)
    df['Rounded_Lng'] = df['Start_Lng'].round(2)
    
    # encode str_features
    le = preprocessing.LabelEncoder()
    df['Weather_Condition'] = le.fit_transform(df['Weather_Condition'])
    
    # add new columns for increased risk with associated conditions and time
    # for now will start by increased risk with associated weather and time
    df['hour'] = df['Start_Time'].astype(str).str.split(' ').str.get(1).str.split(':').str.get(0)
    
    # INCREASED SEVERITY RISK BASED ON LOCATION AND TIME  --------------------------------------------------------------------------
    
    # add col to represent frequency of accidents at this location and time
    df['lt_frequency'] = df.groupby(['Rounded_Lat','Rounded_Lng','hour'])['Severity'].transform('count')

    # add col to represent frequency rank of accidents at this location and time compared to rest of location and times
    # rank 0 = lowest frequency rank
    df['lt_frequency_rank'] = df.groupby(['Rounded_Lat','Rounded_Lng','hour'])['lt_frequency'].rank()

    # add col to represent severity rank of accidents at this location and time compared to rest of location and times
    # rank 0 = lowest frequency rank
    df['lt_severity_rank'] = df.groupby(['Rounded_Lat','Rounded_Lng','hour'])['Severity'].rank()
    
    # INCREASED SEVERITY RISK BASED ON LOCATION AND WEATHER  --------------------------------------------------------------------------
    
    # add col to represent frequency of accidents at this location and time
    df['lw_frequency'] = df.groupby(['Rounded_Lat','Rounded_Lng','Temperature(F)','Wind_Speed(mph)','Precipitation(in)'])['Severity'].transform('count')

    # add col to represent frequency rank of accidents at this location and time compared to rest of location and times
    # rank 0 = lowest frequency rank
    df['lw_frequency_rank'] = df.groupby(['Rounded_Lat','Rounded_Lng','Temperature(F)','Wind_Speed(mph)','Precipitation(in)'])['lw_frequency'].rank()

    # add col to represent severity rank of accidents at this location and time compared to rest of location and times
    # rank 0 = lowest frequency rank
    df['lw_severity_rank'] = df.groupby(['Rounded_Lat','Rounded_Lng','Temperature(F)','Wind_Speed(mph)','Precipitation(in)'])['Severity'].rank()
    
    # drop NaN
    df = df.dropna()
    
    # Keep only relevant columns
    df = df.drop(drop_cols, axis=1)
    
    # for machine learning clean and prep
    if split:
        X=df[predictors]
        y=df[target]
        return X,y

    return df

### MLPRegressor

- sklearn documentation https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

Things to consider:

- lowkey don't know good risk metric
- what do we want to predict?
- we could have 2 different models that relative accident frequency based off of location and weather and location and time (hour) --> predicts lw_frequency_rank and lt_frequency_rank (i think predicting relative accident frequency is more intuitive than relative accident severity)
- we could have 2 different models that relative accident severity based off of location and weather and location and time (hour) --> predicts lw_severity_rank and lt_severity_rank
- we could have 4 different models that do both above ^
- we could come up with a new metric that combines all and only have one model (i just don't know best way to represent that)

In [90]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
train, test = train_test_split(us_accidents, test_size=0.3)

In [104]:
# model to predict relative accident frequency based off of location and time
X, y = prepare_data(df=train, split=True, predictors=['Rounded_Lat','Rounded_Lng','Temperature(F)','Wind_Speed(mph)','Precipitation(in)'], target='lw_frequency_rank')

In [135]:
# also takes forever
X_train, X_test, y_train, y_test = train_test_split(X, y)
lt_regr = MLPRegressor(random_state=1, solver='lbfgs',max_iter=3000, tol=0.02, hidden_layer_sizes=200, alpha=0.005, )
lt_regr.fit(X_train, y_train)
lt_regr.predict(X_test)
lt_regr.score(X_test, y_test) # sad scores :(
# increasing max_iter seems to increase the most?

0.2420844635178483

In [119]:
# this takes forever maybe don't run it
regr = MLPRegressor(random_state=1, solver='lbfgs')
parameters = {"max_iter": list(range(2000,5000,1000)), 
              "hidden_layer_sizes": list(range(100, 500, 100)),
             "tol": [0.1,0.2,0.3,0.4,0.5],
             "alpha": [0.0001, 0.0005, 0.001, 0.005, 0.01],}
cv_scores = [] #holds the cross validation scores
#try every combination of estimators and max depth, recording them and their scores
for mi in parameters['max_iter']:
    for hls in parameters['hidden_layer_sizes']:
        for t in parameters['tol']:
            for a in parameters['alpha']:
                regr.set_params(solver='lbfgs', max_iter=mi, hidden_layer_sizes=hls, tol=t, alpha=a)
                scores = cross_val_score(regr, X_train, y_train, cv=5) #gets cross validation score
                
                cv_scores.append({"max_iter":mi,
                                 "hidden_layer_sizes":hls,
                                 "tol":t,
                                 "alpha":a})

best_params = max(cv_scores, key=lambda x: x["mean_accuracy"]) #find the parameters with the highest (best) cv score
best_params

KeyboardInterrupt: 

In [ ]:
# ideally once best parameters are found and target prediction is confirmed we just predict whatever using nasa power data